# Advanced Schedd Interaction

Launch this tutorial in a Jupyter Notebook on Binder: 
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/htcondor/htcondor-python-bindings-tutorials/master?urlpath=lab/tree/advanced/Advanced-Schedd-Interactions.ipynb)

The introductory tutorial only scratches the surface of what the Python bindings
can do with the ``condor_schedd``; this module focuses on covering a wider range
of functionality:

*  Job and history querying.
*  Advanced job submission.
*  Python-based negotiation with the Schedd.

As usual, we start by importing the relevant modules:

In [1]:
import htcondor
import classad

## Job and History Querying

In [HTCondor Introduction](../introductory/HTCondor-Introduction.ipynb), we covered the `Schedd.xquery` method
and its two most important keywords:

*  ``requirements``: Filters the jobs the schedd should return.
*  ``projection``: Filters the attributes returned for each job.

For those familiar with SQL queries, ``requirements`` performs the equivalent
as the ``WHERE`` clause while ``projection`` performs the equivalent of the column
listing in ``SELECT``.

There are two other keywords worth mentioning:

*  ``limit``: Limits the number of returned ads; equivalent to SQL's ``LIMIT``.
*  ``opts``: Additional flags to send to the schedd to alter query behavior.
   The only flag currently defined is `QueryOpts.AutoCluster`; this
   groups the returned results by the current set of "auto-cluster" attributes
   used by the pool.  It's analogous to ``GROUP BY`` in SQL, except the columns
   used for grouping are controlled by the schedd.

To illustrate these additional keywords, let's first submit a few jobs:

In [2]:
schedd = htcondor.Schedd()
sub = htcondor.Submit({
                       "executable": "/bin/sleep",
                       "arguments":  "5m",
                       "hold":       "True",
                      })
with schedd.transaction() as txn:
    clusterId = sub.queue(txn, 10)
print(clusterId)

7


**Note:** In this example, we used the ``hold`` submit command to indicate that
the jobs should start out in the ``condor_schedd`` in the *Hold* state; this
is used simply to prevent the jobs from running to completion while you are
running the tutorial.

We now have 10 jobs running under ``clusterId``; they should all be identical:

In [3]:
print(sum(1 for _ in schedd.xquery(projection=["ProcID"], requirements="ClusterId==%d" % clusterId, limit=5)))

5


The ``sum(1 for _ in ...)`` syntax is a simple way to count the number of items
produced by an iterator without buffering all the objects in memory.

## Querying many Schedds

On larger pools, it's common to write Python scripts that interact with not one but many schedds.  For example,
if you want to implement a "global query" (equivalent to ``condor_q -g``; concatenates all jobs in all schedds),
it might be tempting to write code like this:

In [4]:
jobs = []
for schedd_ad in htcondor.Collector().locateAll(htcondor.DaemonTypes.Schedd):
    schedd = htcondor.Schedd(schedd_ad)
    jobs += schedd.xquery()
print(len(jobs))

10


This is sub-optimal for two reasons:

*  ``xquery`` is not given any projection, meaning it will pull all attributes for all jobs -
   much more data than is needed for simply counting jobs.
*  The querying across all schedds is serialized: we may wait for painfully long on one or two
   "bad apples."

We can instead begin the query for all schedds simultaneously, then read the responses as
they are sent back.  First, we start all the queries without reading responses:

In [5]:
queries = []
coll_query = htcondor.Collector().locateAll(htcondor.DaemonTypes.Schedd)
for schedd_ad in coll_query:
    schedd_obj = htcondor.Schedd(schedd_ad)
    queries.append(schedd_obj.xquery())

The iterators will yield the matching jobs; to return the autoclusters instead of jobs, use
the ``AutoCluster`` option (``schedd_obj.xquery(opts=htcondor.QueryOpts.AutoCluster)``).  One
auto-cluster ad is returned for each set of jobs that have identical values for all significant
attributes.  A sample auto-cluster looks like:

       [
        RequestDisk = DiskUsage;
        Rank = 0.0;
        FileSystemDomain = "hcc-briantest7.unl.edu";
        MemoryUsage = ( ( ResidentSetSize + 1023 ) / 1024 );
        ImageSize = 1000;
        JobUniverse = 5;
        DiskUsage = 1000;
        JobCount = 1;
        Requirements = ( TARGET.Arch == "X86_64" ) && ( TARGET.OpSys == "LINUX" ) && ( TARGET.Disk >= RequestDisk ) && ( TARGET.Memory >= RequestMemory ) && ( ( TARGET.HasFileTransfer ) || ( TARGET.FileSystemDomain == MY.FileSystemDomain ) );
        RequestMemory = ifthenelse(MemoryUsage isnt undefined,MemoryUsage,( ImageSize + 1023 ) / 1024);
        ResidentSetSize = 0;
        ServerTime = 1483758177;
        AutoClusterId = 2
       ]

We use the `poll` function, which will return when a query has available results:

In [6]:
job_counts = {}
for query in htcondor.poll(queries):
    schedd_name = query.tag()
    job_counts.setdefault(schedd_name, 0)
    count = len(query.nextAdsNonBlocking())
    job_counts[schedd_name] += count
    print("Got {} results from {}.".format(count, schedd_name))
print(job_counts)

Got 10 results from jovyan@63a78c05ff13.
{'jovyan@63a78c05ff13': 10}


The `QueryIterator.tag` method is used to identify which query is returned; the
tag defaults to the Schedd's name but can be manually set through the ``tag`` keyword argument
to `Schedd.xquery`.

## History Queries

After a job has finished in the Schedd, it moves from the queue to the history file.  The
history can be queried (locally or remotely) with the `Schedd.history` method:

In [7]:
schedd = htcondor.Schedd()
for ad in schedd.history('true', ['ProcId', 'ClusterId', 'JobStatus', 'WallDuration'], 2):
    print(ad)


    [
        JobStatus = 3; 
        ProcId = 9; 
        ClusterId = 5
    ]

    [
        JobStatus = 3; 
        ProcId = 4; 
        ClusterId = 6
    ]


At the time of writing, unlike `Schedd.xquery`, `Schedd.history`
takes positional arguments and not keyword.  The first argument a job constraint; second is the
projection list; the third is the maximum number of jobs to return.

## Advanced Job Submission

In [HTCondor Introduction](../introductory/HTCondor-Introduction.ipynb), we introduced the `Submit` object.  `Submit`
allows jobs to be created using the *submit file* language.  This is the well-documented, familiar
means for submitting jobs via ``condor_submit``.  This is the preferred mechansim for submitting
jobs from Python.

Internally, the submit files are converted to a job ClassAd.  The older `Schedd.submit`
method allows jobs to be submitted as ClassAds.  For example:

In [8]:
import os

schedd = htcondor.Schedd()
job_ad = classad.ClassAd({
     'Cmd': '/bin/sh',
     'JobUniverse': 5,
     'Iwd': os.path.abspath("/tmp"),
     'Out': 'testclaim.out',
     'Err': 'testclaim.err',
     'Arguments': 'sleep 5m',
})
clusterId = schedd.submit(job_ad, count=2)
print(clusterId)

8


This will submit two copies of the job described by ``job_ad`` into a single job cluster.

**Hint**: To generate an example ClassAd, take a sample submit description
file and invoke:

      condor_submit -dump <filename> [cmdfile]

Then, load the resulting contents of ``<filename>`` into Python.

Calling `Schedd.submit` standalone will automatically create and commit a transaction.
Multiple jobs can be submitted atomically and more efficiently within a `Schedd.transaction()`
context.

Each `Schedd.submit` invocation will create a new job cluster; all attributes will be
identical except for the ``ProcId`` attribute (process IDs are assigned in monotonically increasing order,
starting at zero).  If jobs in the same cluster need to differ on additional attributes, one may use the
`Schedd.submitMany` method:

In [9]:
foo = classad.ClassAd({'myAttr': 'foo'})
bar = classad.ClassAd({'myAttr': 'bar'})
clusterId = schedd.submitMany(job_ad, [(foo, 2), (bar, 2)])
print(clusterId)

9


In [10]:
query = schedd.xquery('ClusterId=={}'.format(clusterId), ['ProcId', 'myAttr'])
for ad in query:
    print(ad)


    [
        ServerTime = 1589900105; 
        ProcId = 0; 
        myAttr = "foo"
    ]

    [
        ServerTime = 1589900105; 
        ProcId = 1; 
        myAttr = "foo"
    ]

    [
        ServerTime = 1589900105; 
        ProcId = 2; 
        myAttr = "bar"
    ]

    [
        ServerTime = 1589900105; 
        ProcId = 3; 
        myAttr = "bar"
    ]


`Schedd.submitMany` takes a basic job ad (sometimes referred to as the *cluster ad*),
shared by all jobs in the cluster and a list of *process ads*.  The process ad list indicates
the attributes that should be overridden for individual jobs, as well as the number of such jobs
that should be submitted.

## Job Spooling

HTCondor file transfer will move output and input files to and from the submit host; these files will
move back to the original location on the host.  In some cases, this may be problematic; you may want
to submit one set of jobs to run ``/home/jovyan/a.out``, recompile the binary, then submit a fresh
set of jobs.  By using the *spooling* feature, the ``condor_schedd`` will make a private copy of
``a.out`` after submit, allowing the user to make new edits.

**Hint**: Although here we give an example of using `Schedd.spool` for spooling on
   the local Schedd, with appropriate authoriation the same methods can be used for submitting to
   remote hosts.

To spool, one must specify this at submit time and invoke the `Schedd.spool` method
and provide an ``ad_results`` array:

In [11]:
ads = []
cluster = schedd.submit(job_ad, 1, spool=True, ad_results=ads)
schedd.spool(ads)
print(ads)

[[ ClusterId = 10; LocalSysCpu = 0.0; RemoteSysCpu = 0.0; OnExitRemove = true; PeriodicRemove = false; LocalUserCpu = 0.0; RemoteUserCpu = 0.0; StreamErr = false; HoldReason = "Spooling input data files"; CommittedSuspensionTime = 0; ProcId = 0; PeriodicHold = false; CondorPlatform = "$CondorPlatform: X86_64-CentOS_5.11 $"; ExitStatus = 0; ShouldTransferFiles = "YES"; LastSuspensionTime = 0; NumJobStarts = 0; NumCkpts = 0; WhenToTransferOutput = "ON_EXIT"; TargetType = "Machine"; JobNotification = 0; BufferSize = 524288; ImageSize = 100; PeriodicRelease = false; CompletionDate = 0; RemoteWallClockTime = 0.0; Arguments = "sleep 5m"; WantCheckpoint = false; NumSystemHolds = 0; CumulativeSuspensionTime = 0; QDate = 1589900105; EnteredCurrentStatus = 1589900105; CondorVersion = "$CondorVersion: 8.9.6 Apr 07 2020 BuildID: UW_Python_Wheel_Build $"; MyType = "Job"; Owner = undefined; ExitBySignal = false; JobUniverse = 5; BufferBlockSize = 32768; Err = "testclaim.err"; NiceUser = false; CoreS

This will copy the files into the Schedd's ``spool`` directory.  After the job completes, its
output files will stay in the spool.  One needs to call `Schedd.retrieve` to
move the outputs back to their final destination:

In [12]:
schedd.retrieve("ClusterId=={}".format(cluster))

Finally, let's clean up after ourselves (this will remove all of the jobs you own from the queue).

In [13]:
import getpass

schedd.act(htcondor.JobAction.Remove, f'Owner=="{getpass.getuser()}"')

[ TotalJobAds = 15; TotalPermissionDenied = 0; TotalAlreadyDone = 0; TotalNotFound = 0; TotalSuccess = 13; TotalChangedAds = 1; TotalBadStatus = 0; TotalError = 0 ]